In [5]:
import pandas as pd
review=pd.read_csv('Hotel_Reviews.csv')
review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [40]:
review = review.groupby('Hotel_Name').agg({'Negative_Review': ', '.join,'Positive_Review': ', '.join}).reset_index()

In [41]:
review.head()

,Hotel_Name,Negative_Review,Positive_Review
0,11 Cadogan Gardens,Thought the prise of drinks at the bar a litt...,We were particularly impressed by the very wa...
1,1K Hotel,Air conditioning in room didn t work and desp...,Location good close to le Marais and 3e arron...
2,25hours Hotel beim MuseumsQuartier,Breakfast not included and buffet really expe...,Cool vintage style in the middle of the museu...
3,41,"There wasn t a thing that we didn t like , No...",Its central proximity close to all services a...
4,45 Park Lane Dorchester Collection,More kinds of fruit juice will make the mini ...,Everything here are almost perfect the staffs...


In [42]:
review['review_text'] = review['Positive_Review'].astype(str) + review['Negative_Review'].astype(str)

In [43]:
review.head()

,Hotel_Name,Negative_Review,Positive_Review,review_text
0,11 Cadogan Gardens,Thought the prise of drinks at the bar a litt...,We were particularly impressed by the very wa...,We were particularly impressed by the very wa...
1,1K Hotel,Air conditioning in room didn t work and desp...,Location good close to le Marais and 3e arron...,Location good close to le Marais and 3e arron...
2,25hours Hotel beim MuseumsQuartier,Breakfast not included and buffet really expe...,Cool vintage style in the middle of the museu...,Cool vintage style in the middle of the museu...
3,41,"There wasn t a thing that we didn t like , No...",Its central proximity close to all services a...,Its central proximity close to all services a...
4,45 Park Lane Dorchester Collection,More kinds of fruit juice will make the mini ...,Everything here are almost perfect the staffs...,Everything here are almost perfect the staffs...


In [45]:
#findout similarity between the reviews of hotel using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
review['review_text'] = review['review_text'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(review['review_text'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape
#You see that over 80,000 different words were used to describe the 1492 hotels in dataset.

(1492, 81002)

In [46]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [49]:
indices = pd.Series(review.index, index=review['Hotel_Name']).drop_duplicates()

In [52]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the hotel that matches the hotel_name
    idx = indices[title]

    # Get the pairwsie similarity scores of all hotels with that hotel
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the hotels based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar hotels
    sim_scores = sim_scores[1:11]

    # Get the hotel indices
    hotel_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar hotel
    return review['Hotel_Name'].iloc[hotel_indices]

In [53]:
#user input hotel name 11 cadogan garden and get top 10 similar hotel based on user review
get_recommendations('11 Cadogan Gardens')

330                         Flemings Mayfair
1316                    The Bloomsbury Hotel
1393                   The Royal Horseguards
1323                The Chesterfield Mayfair
1368             The Montague On The Gardens
1319                    The Cavendish London
1175        Radisson Blu Edwardian Hampshire
1384                    The Principal London
1370                The Montcalm Marble Arch
1177    Radisson Blu Edwardian Mercer Street
Name: Hotel_Name, dtype: object

In [54]:
get_recommendations('XO Hotel')

160     Best Western Premier Kapital Op ra
752          Hotel Saint Petersbourg Opera
708                         Hotel Monsieur
416                       H tel Diva Opera
720                 Hotel Op ra Richepanse
1071                       Newhotel Roblin
399                          H tel Bedford
344                    Gardette Park Hotel
422              H tel Exquis by Elegancia
165       Best Western Premier Op ra Li ge
Name: Hotel_Name, dtype: object